In [1]:
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/functions.py

!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/risks.txt
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/threats.txt

--2021-04-11 22:31:28--  https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6913 (6.8K) [text/plain]
Saving to: ‘functions.py’

functions.py        100%[===================>]   6.75K  --.-KB/s    in 0s      

2021-04-11 22:31:28 (91.0 MB/s) - ‘functions.py’ saved [6913/6913]

--2021-04-11 22:31:28--  https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/risks.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2675 (2.

In [2]:
import functions
from functions import *
import spacy
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

threats_file = '/content/threats.txt'
threats_dict = terms_dict(threats_file)

risks_file = '/content/risks.txt'
risks_dict = terms_dict(risks_file)



# Añadimos el termino 'intentional' a todos los términos que tengan la palabra 
# 'deliberated' porque 'deliberated' da problemas y sugerimos evitar esa palabra
for k,v in threats_dict.items():
  if 'Deliberated' in threats_dict[k]:
    add_word_to_dict(threats_dict,k, 'intentional')

# añadimos las palabras para el acrónimo HW
for k,v in threats_dict.items():
  if 'HW' in threats_dict[k]:
    add_word_to_dict(threats_dict,k, 'hardware')

# añadimos las palabras para el acrónimo SW
for k,v in threats_dict.items():
  if 'SW' in threats_dict[k]:
    add_word_to_dict(threats_dict,k, 'software')

# risks_dict

In [10]:
def process_threats_risks_sentence(sentence: str, result: {}):
  """Function that process a sentence for finding the terms prob, anomalyType, 
  threatType and impact""" 
  
  # sentence = sentence.replace(",",".") # Para admitir decimales con "," o "."

  doc = nlp(str(sentence))

  raw_risks = extract_dobject(sentence)
  risks= []
  aux = []
  for element in raw_risks:
    if element == "," or element == "and":
      risks.append(aux)
      aux = []
    elif element == raw_risks[-1]:
      risks.append(aux)
    else:
      aux.append(element)

  cont = 0
  for risk in risks:
    cont += 1
    risk = clean(risk)
    risk_matches = search_matches(risks_dict, risk)
    risk_count = counter(risk_matches)
    risk = higher_frecuency_term(risk_count)
    result["riskType"+str(cont)] = risk

  threat = extract_subject(sentence)
  threat = clean(threat)
  threat_matches = search_matches(threats_dict, threat)  
  threat_count = counter(threat_matches)  
  threat = higher_frecuency_term(threat_count)  
  result["threatType"] = threat

  # result["prob"] = extract_prob(sentence)
  # result["impact"] = extract_impact(sentence)
    
  return result
  # print(doc)
  # print(result)

In [12]:
result = {
  }

sentence = "corporate brand image damages threat may generate an accident risk, fire risk, flooding risk, natural disaster risk and terrorism attack risk"
# importante terminar la enumeración con "and"

result = process_threats_risks_sentence(sentence, result)
result = {"result": [result]}
write_json("results.json", result) # escribir archivo con los resultados
result

{'result': [{'riskType1': 'AccidentRisk',
   'riskType2': 'FireRisk',
   'riskType3': 'FloodingRisk',
   'riskType4': 'NaturalDisasterRisk',
   'riskType5': 'TerrorismAttackRisk',
   'threatType': 'CorporateBrandImageDamages'}]}

In [5]:
!wget https://raw.githubusercontent.com/tylerneylon/explacy/master/explacy.py

--2021-04-11 22:31:33--  https://raw.githubusercontent.com/tylerneylon/explacy/master/explacy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6896 (6.7K) [text/plain]
Saving to: ‘explacy.py’

explacy.py          100%[===================>]   6.73K  --.-KB/s    in 0s      

2021-04-11 22:31:33 (83.0 MB/s) - ‘explacy.py’ saved [6896/6896]



In [6]:
import explacy

sentence = "corporate brand image damages threat may generate an accident risk, fire risk, flooding risk, natural disaster risk, terrorism attack"

nlp = spacy.load('en')

explacy.print_parse_info(nlp, sentence)

# for sentence in sentences:
#   explacy.print_parse_info(nlp, sentence)

Dep tree            Token     Dep type Lemma     Part of Sp
─────────────────── ───────── ──────── ───────── ──────────
                ┌─► corporate amod     corporate ADJ       
             ┌─►└── brand     compound brand     NOUN      
             │  ┌─► image     compound image     NOUN      
          ┌─►└──┴── damages   compound damage    NOUN      
       ┌─►└──────── threat    nsubj    threat    NOUN      
       │        ┌─► may       aux      may       VERB      
┌──────┴────────┴── generate  ROOT     generate  VERB      
│              ┌──► an        det      an        DET       
│              │┌─► accident  compound accident  NOUN      
└─►┌───────────┴┼── risk      dobj     risk      NOUN      
   │            └─► ,         punct    ,         PUNCT     
   │            ┌─► fire      compound fire      NOUN      
   └─►┌┬────────┼── risk      conj     risk      NOUN      
      ││        └─► ,         punct    ,         PUNCT     
      ││        ┌─► flooding  compound f